In [1]:
import json
import pandas as pd
import os
import numpy as np

In [2]:
def extract_txo(targets):
    tx_die = targets[targets.Index == 'TxDie']
    tx_succ = targets[targets.Index == 'TxSucc']

    txo = dict()

    sel = tx_die[tx_die.Tag == 'Pub']
    txi = sel.N.sum()
    txd = (sel.M * sel.N).sum()

    sel = tx_succ[tx_succ.Tag == 'Pub']
    txs = (sel.M * sel.N).sum()
    txl = txi - txs - txd

    txo['pub'] = {'txi': txi, 'txd': txd, 'txs': txs, 'txl': txl}

    sel = tx_die[tx_die.Tag == 'Pri']
    txi = sel.N.sum()
    txd = (sel.M * sel.N).sum()

    sel = tx_succ[tx_succ.Tag == 'Pri']
    txs = (sel.M * sel.N).sum()
    txl = txi - txs - txd

    txo['eng'] = {'txi': txi, 'txd': txd, 'txs': txs, 'txl': txl}
    return txo


def attach_pars_tx(ps, txo):
    ps_pub = np.array([txo['pub'][o] for o in ['txs', 'txd', 'txl']]) / txo['pub']['txi']
    tx_eng = np.array([txo['eng'][o] for o in ['txs', 'txd', 'txl']]) / txo['eng']['txi']

    for p in js['pars']:
        txi_ep = txo['eng']['txi'] * p['p_pri_on_pub'] 
        tx_eng_ei = np.array([txo['eng'][o] for o in ['txs', 'txd', 'txl']]) - txi_ep * ps_pub
        tx_eng_ei[tx_eng_ei < 0] = 0
        ps_pri = tx_eng_ei / (txo['eng']['txi'] * (1 - p['p_pri_on_pub']))

        p['r_txs_pub'], p['r_txd_pub'], p['r_txl_pub'] = ps_pub * 0.5
        p['r_txs_pri'], p['r_txd_pri'], p['r_txl_pri'] = p['dur_pri'] * ps_pri
        
        if p['r_txd_pri'] < p['r_txd_pub']:
            diff = p['r_txd_pub'] - p['r_txd_pri']
            p['r_txd_pri'] = p['r_txd_pub']
            p['r_txl_pri'] -= diff
            
        if p['r_txs_pri'] > p['r_txs_pub']:
            diff = p['r_txs_pri'] - p['r_txs_pub']
            p['r_txs_pri'] = p['r_txs_pub']
            p['r_txl_pri'] += diff

In [3]:
locations = list(pd.read_csv('data/locations.csv').Location)
locations

['India',
 'Andhra_Pradesh',
 'Arunachal_Pradesh',
 'Assam',
 'Bihar',
 'Chandigarh',
 'Chhattisgarh',
 'Delhi',
 'Gujarat',
 'Haryana',
 'Himachal_Pradesh',
 'Jammu_and_Kashmir',
 'Jharkhand',
 'Karnataka',
 'Kerala',
 'Madhya_Pradesh',
 'Maharashtra',
 'Manipur',
 'Meghalaya',
 'Nagaland',
 'Odisha',
 'Punjab',
 'Rajasthan',
 'Tamil_Nadu',
 'Telangana',
 'Uttar_Pradesh',
 'Uttarakhand',
 'West_Bengal']

In [4]:
for loc in locations:
    js = json.load(open(f'data/{loc}/pars_under.json', 'r'))
    targets = pd.read_csv(f'data/{loc}/targets.csv')
    txo = extract_txo(targets)
    attach_pars_tx(js['pars'], txo)
    
    with open(f'data/{loc}/pars_txo.json', 'w') as f:
        json.dump(js, f)